In [1]:
import os, sys
import numpy as np
import matplotlib.pyplot as plt
import pickle
from copy import deepcopy
import CBcm

In [2]:
from scipy import stats

import iminuit

In [3]:
sys.path.append('/home/jp/projects/atm_nu/mceq_tysak/muon_fit/modules')

In [4]:
common_settings = pickle.load(open('/home/jp/projects/atm_nu/ddm_results/final_settings/common_settings_splitVHE.pkl'))

In [5]:
import mufit
reload(mufit)
import mufit_plots as mfp
reload(mfp)

MUFIT: Using quickflux2!
MUFIT: Using quickflux2!
MufitPlots does not use the mceq flux evaluator directly
MufitPlots does not use the mceq flux evaluator directly


<module 'mufit_plots' from '/home/jp/projects/atm_nu/mceq_tysak/muon_fit/modules/mufit_plots.pyc'>

In [6]:
fitname = 'prd22_daemon_v2np'
filename = '/home/jp/projects/atm_nu/ddm_results/'+fitname+'.pkl'


In [7]:
fit = mufit.MuFit(datasets = ['bessfull',
                              'l3c', 
                              'opera', 
                              'minos',
                              'cms',
                              'deis',
                              'mutronratio',
                             'deisratio'],

                 flux_splines = 'daemon-v2n',)


/home/jp/projects/atm_nu/mceq_tysak/muon_fit/modules/resources/daemonsplines_20230204_inclusertables.pkl
Excluded params removed from known_params.
No calibration file for daemon-v2n
Adjusting DEIS name
daemon-v2n
('Using errors in mode: ', 'mixed')
MUTRON ratio: adding ALL errors in quadrature
DEIS ratio: adding ALL errors in quadrature


In [9]:
physics_params = fit.mceq.known_params
print(physics_params)

[u'K+_158G', u'K+_2P', u'K+_31G', u'K-_158G', u'K-_2P', u'K-_31G', u'n_158G', u'n_2P', u'p_158G', u'p_2P', u'pi+_158G', u'pi+_20T', u'pi+_2P', u'pi+_31G', u'pi-_158G', u'pi-_20T', u'pi-_2P', u'pi-_31G', u'GSF_1', u'GSF_2', u'GSF_3', u'GSF_4', u'GSF_5', u'GSF_6']


In [13]:
my_flux_parameters = {}
for onep in physics_params:
    if 'GSF' in onep:
        my_flux_parameters[onep] = [0., False, -4, 4, 0., 1]
    elif ('31G' in onep) or ('158G' in onep):
        my_flux_parameters[onep] = [0., False, -10, 10, 0., 1]
    elif ('20T' in onep) or ('2P' in onep):
        if 'pi+' in onep:
            my_flux_parameters[onep] = [-0.3, False, -5., 5.,  0.,  2.]
        elif 'pi-' in onep:
            my_flux_parameters[onep] = [0,False, -33,  33,    0,  10]
        elif 'K+' in onep:
            my_flux_parameters[onep]= [ 0., False, -19,  19,  0,  10]
        elif 'K-' in onep:
            my_flux_parameters[onep]=[-20., False, -25,  30, 0.,  20.]
        elif 'p' in onep:
            my_flux_parameters[onep]=[12., False, -16, 16,  0., 6]
        elif 'n' in onep:
            my_flux_parameters[onep]=[0., False, -6, 6,   0,  2.5]
for onep in physics_params:
    print(onep, my_flux_parameters[onep])

(u'K+_158G', [0.0, False, -10, 10, 0.0, 1])
(u'K+_2P', [0.0, False, -19, 19, 0, 10])
(u'K+_31G', [0.0, False, -10, 10, 0.0, 1])
(u'K-_158G', [0.0, False, -10, 10, 0.0, 1])
(u'K-_2P', [-20.0, False, -25, 30, 0.0, 20.0])
(u'K-_31G', [0.0, False, -10, 10, 0.0, 1])
(u'n_158G', [0.0, False, -10, 10, 0.0, 1])
(u'n_2P', [0.0, False, -6, 6, 0, 2.5])
(u'p_158G', [0.0, False, -10, 10, 0.0, 1])
(u'p_2P', [12.0, False, -16, 16, 0.0, 6])
(u'pi+_158G', [0.0, False, -10, 10, 0.0, 1])
(u'pi+_20T', [-0.3, False, -5.0, 5.0, 0.0, 2.0])
(u'pi+_2P', [-0.3, False, -5.0, 5.0, 0.0, 2.0])
(u'pi+_31G', [0.0, False, -10, 10, 0.0, 1])
(u'pi-_158G', [0.0, False, -10, 10, 0.0, 1])
(u'pi-_20T', [0, False, -33, 33, 0, 10])
(u'pi-_2P', [0, False, -33, 33, 0, 10])
(u'pi-_31G', [0.0, False, -10, 10, 0.0, 1])
(u'GSF_1', [0.0, False, -4, 4, 0.0, 1])
(u'GSF_2', [0.0, False, -4, 4, 0.0, 1])
(u'GSF_3', [0.0, False, -4, 4, 0.0, 1])
(u'GSF_4', [0.0, False, -4, 4, 0.0, 1])
(u'GSF_5', [0.0, False, -4, 4, 0.0, 1])
(u'GSF_6', [0.0

In [14]:
mysyst = deepcopy(fit.systematics)
mysyst.update({# Fixing DEIS because errors are included by default
               'deis_mscat':  [0.0, True, -3, 3],
               'deis_bfield':  [0.0, True, -3, 3],
               'deis_norm':  [0.0, True, -3, 3],

               # # Added for testing
#                    'amsratio_mc': 0., # np.random.uniform(-3,3),
#                    'amsratio_statistics':  0., #np.random.uniform(-3,3),
#                    'amsratio_protons':  0., #np.random.uniform(-3,3),

               # L3c (all)
               'l3c_align': 0.0042, #np.random.uniform(-3,3),
               'l3c_bfield': -3.30, #np.random.uniform(-3,3),
               'l3c_eloss': -2.92, #np.random.uniform(-3,3),
               'l3c_overb': -0.5, #np.random.uniform(-3,3),
               'l3c_normHor': -1.41, #np.random.uniform(-3,3),
               'l3c_normVert': -1.591, #np.random.uniform(-3,3),
               'l3c_pres': 1.352, #np.random.uniform(-3,3),

               'mn_escale': -1.59, #np.random.uniform(-3,3),

               # Bess
               'bess_align': -1.72, #np.random.uniform(-3,3),
               'bess_finres': 0.2, #np.random.uniform(-3,3),
               })

In [15]:
nfits = 1
counter = 0
bestfit = 1E9
defprior = 60
for ifit in range(nfits):
    print 'Internal fit ', ifit
    one_m, one_f = fit.run_fit(
        flux_parameters = my_flux_parameters,        
        exp_parameters = mysyst,
        fit_data = 'all',
        fit_mode = 'physics',
        use_priors = True,
        print_mode = False,
        dry_run = False,
        emin=5.,
        fake_data = False,
        )
    
    this_fit = fit.results['fmin']['fval']
    print 'This fit', this_fit
    if this_fit < bestfit:
        print 'Updated results for fit', this_fit, bestfit
        bestfit = deepcopy(this_fit)
        results = deepcopy(fit.results)
        m = one_m
        f = one_f

Internal fit  0


/home/jp/projects/atm_nu/mceq_tysak/muon_fit/modules/mufit.py:303: RuntimeWarning: invalid value encountered in divide
  phys_priors_mean)**2/phys_priors_err**2)[prior_bool]


Successful fit!
This fit 498.332661636
Updated results for fit 498.332661636 1000000000.0


In [16]:
free_params = 0
for one_param in fit.fit_keys:
    if not fit.minuit_settings['fix_'+one_param]:
        free_params += 1
#print(free_params, len(fit.fit_keys))

In [17]:
all_keys = results['params'].keys()
dof = fit.results['bins']-free_params
print 'Chi2',fit.results['chi2'], '/', dof, 'dof (approximate)'
p_value = 1-stats.chi2.cdf(fit.results['chi2'], dof)
print('P-value', p_value)
syst_keys = []
print '\t'.join(['Param', 'value','error'])
for one_param in physics_params:
    all_keys.remove(one_param)
    print '\t'.join([one_param, 
                     "%0.2f" % results['params'][one_param]['value'],
                    "%0.2f" % results['params'][one_param]['error']])
all_keys.sort()
for one_param in all_keys:
    print '\t'.join([one_param, 
                     "%0.2f" % results['params'][one_param]['value'],
                    "%0.2f" % results['params'][one_param]['error']])



Chi2 410.9381909662357 / 424 dof (approximate)
('P-value', 0.6665038648781165)
Param	value	error
K+_158G	0.28	0.98
K+_2P	4.62	4.82
K+_31G	-1.96	0.76
K-_158G	0.26	0.99
K-_2P	-22.95	10.99
K-_31G	-0.29	0.99
n_158G	2.03	0.79
n_2P	3.67	2.24
p_158G	0.32	0.95
p_2P	12.78	5.12
pi+_158G	0.13	0.10
pi+_20T	-1.90	0.33
pi+_2P	-4.54	1.50
pi+_31G	-2.21	0.78
pi-_158G	4.07	0.70
pi-_20T	-5.65	2.19
pi-_2P	-2.15	7.28
pi-_31G	-3.26	0.38
GSF_1	0.88	0.89
GSF_2	0.37	0.99
GSF_3	0.12	0.99
GSF_4	0.59	0.99
GSF_5	-2.19	0.95
GSF_6	0.56	0.98
bess_align	-1.67	0.67
bess_finres	0.16	0.82
deis_bfield	0.00	0.50
deis_mscat	0.00	0.50
deis_norm	0.00	0.50
l3c_align	-0.02	0.07
l3c_bfield	-2.99	0.78
l3c_eloss	-2.41	0.79
l3c_normHor	-1.28	0.55
l3c_normVert	-1.17	0.52
l3c_overb	-0.20	0.49
l3c_pres	1.20	0.45
mn_escale	-1.65	0.32


In [18]:
fit.results['flux_parameters'] = my_flux_parameters

In [19]:
fit.results['flux_tables']

'daemon-v2n'

In [20]:
pickle.dump(fit.results, open(filename, 'wb'))
print('Written', filename)

('Written', '/home/jp/projects/atm_nu/ddm_results/prd22_daemon_v2np.pkl')


In [17]:
m.matrix(correlation=True)

----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|              |        he_K-        he_K+     vhe2_pi-       he_pi+       vhe3_n      vhe3_K-      vhe3_K+     vhe2_pi+       vhe3_p       le_pi+       le_pi-       he_pi-        le_K+        le_K-        GSF_4        GSF_5        GSF_6        GSF_1        GSF_2        GSF_3     vhe3_pi+         he_p     vhe3_pi-         he_n   l3c_bfield    l3c_overb    mn_escale   bess_align l3c_normVert  l3c_normHor    l3c_eloss  bess_finres     l3c_pres    l3c_align |
----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|        he_K- |        1.000        0.015       -0.003        0.045        0.014       -0.045        0.025        0.003       -0.001        0.002        0.000       -0.080        0.002        0.001       -0.002        0.004        0.001       -0.018       -0.001       -0.000       -0.018       -0.013        0.044       -0.028        0.006       -0.005        0.008        0.015        0.004       -0.001        0.003       -0.003       -0.000        0.004 |
|        he_K+ |        0.015        1.000        0.002       -0.280       -0.027        0.087       -0.082       -0.005        0.004       -0.015       -0.017        0.029       -0.016        0.000        0.002       -0.005       -0.002       -0.014        0.002        0.000        0.045        0.006       -0.093       -0.013        0.023        0.019       -0.019        0.018        0.007        0.003        0.016        0.002        0.022        0.020 |
|     vhe2_pi- |       -0.003        0.002        1.000        0.498       -0.362       -0.432       -0.271        0.872       -0.339        0.010        0.079        0.286       -0.018       -0.004        0.000       -0.053       -0.025       -0.557        0.010        0.000       -0.082        0.043        0.161        0.077        0.067        0.078        0.013        0.016        0.034        0.043        0.058        0.010       -0.124       -0.033 |
|       he_pi+ |        0.045       -0.280        0.498        1.000       -0.102       -0.048        0.133        0.372       -0.217        0.049        0.114        0.720       -0.038        0.010       -0.005        0.013        0.016       -0.424       -0.002        0.003        0.060       -0.276        0.140       -0.114        0.033       -0.118        0.005        0.186        0.049        0.006        0.012       -0.049       -0.079       -0.028 |
|       vhe3_n |        0.014       -0.027       -0.362       -0.102        1.000        0.077       -0.022       -0.108       -0.125        0.037        0.089       -0.044        0.049        0.003       -0.004        0.018       -0.005       -0.038       -0.003       -0.001        0.063       -0.023       -0.086       -0.113       -0.047       -0.013       -0.015       -0.061       -0.022       -0.012       -0.036        0.006       -0.003       -0.016 |
|      vhe3_K- |       -0.045        0.087       -0.432       -0.048        0.077        1.000        0.894       -0.341        0.055       -0.022       -0.071        0.006       -0.013       -0.001        0.024       -0.099        0.040       -0.087        0.017        0.004       -0.080        0.003       -0.

In [18]:
fit.results['cov_params'] = m.list_of_vary_param()
ordered_matrix = np.zeros_like(fit.results['corr_matrix'])
# Need to reorder this business
syst_params = ['l3c_bfield','l3c_overb','l3c_normVert', 'bess_align','bess_finres',
               'l3c_normHor','l3c_eloss','l3c_pres','l3c_align', 'mn_escale']
syst_params.sort()
ordered_phys = np.zeros(len(physics_params))
ordered_syst = np.zeros(len(syst_params))
for ip, one_param in enumerate(physics_params):
    ordered_phys[ip] = fit.results['cov_params'].index(one_param)
for ip, one_param in enumerate(syst_params):
    ordered_syst[ip] = fit.results['cov_params'].index(one_param)
    
full_list = physics_params + syst_params
full_in = np.array(np.concatenate((ordered_phys, ordered_syst)),dtype=int)
for j, one_param in enumerate(full_list):
    for k in range(len(full_in)):
        ordered_matrix[j,k] = fit.results['corr_matrix'][full_in[j], full_in[k]]

In [19]:
if not os.path.exists('/home/jp/projects/atm_nu/ddm_results/'+fitname):
    os.mkdir('/home/jp/projects/atm_nu/ddm_results/'+fitname)

In [20]:
maxb = len(physics_params)
corr_matrix = deepcopy(ordered_matrix)

for i in range(corr_matrix.shape[0]):
    for j in range(corr_matrix.shape[0]):
        if i<j: corr_matrix[i,j]=0.

params = deepcopy(full_list)

# Adjust parameter names
for i, onep in enumerate(params):
    params[i] = common_settings['param_legend'][onep]

    
fig = plt.figure(figsize=(12*0.7,10*0.7))
ax = fig.add_subplot(111)
xmarks = np.arange(0.5, corr_matrix.shape[0], 1.)
plt.pcolor(corr_matrix[:maxb,:maxb], cmap='RdBu', vmin=-1, vmax=1)
plt.xticks(xmarks, params[:maxb], rotation=90)
plt.yticks(xmarks, params[:maxb])
plt.ylim(0,maxb)
plt.xlim(0,maxb)
ax.set_ylim(ax.get_ylim()[::-1])
ax.xaxis.tick_top()
plt.colorbar()
plt.tight_layout()



#fig.savefig('/home/jp/projects/atm_nu/ddm_results/'+fitname+'/corr_phys_noscale_qf2.png', dpi=300)
#fig.savefig('/home/jp/projects/atm_nu/ddm_results/'+fitname+'/corr_phys_noscale_qf2.pdf')


KeyError: 'vhe2_pi+'

In [ ]:
maxb = None
corr_matrix = deepcopy(ordered_matrix)

for i in range(corr_matrix.shape[0]):
    for j in range(corr_matrix.shape[0]):
        if i<j: corr_matrix[i,j]=0.

params = deepcopy(full_list)

# Adjust parameter names
for i, onep in enumerate(params):
    params[i] = common_settings['param_legend'][onep]
    
fig = plt.figure(figsize=(14,10))
ax = fig.add_subplot(111)
xmarks = np.arange(0.5, corr_matrix.shape[0], 1.)
plt.pcolor(corr_matrix[:maxb,:maxb], cmap='RdBu', vmin=-1, vmax=1)
plt.xticks(xmarks, params[:maxb], rotation=90)
plt.yticks(xmarks, params[:maxb])
plt.ylim(0,maxb)
plt.xlim(0,maxb)
ax.set_ylim(ax.get_ylim()[::-1])
ax.xaxis.tick_top()
plt.colorbar()
#fig.savefig('/home/jp/projects/atm_nu/ddm_results/prd22_results_may9_v12_final/corr_full.png', dpi=300)
#fig.savefig('/home/jp/projects/atm_nu/ddm_results/prd22_results_may9_v12_final/corr_full.pdf')


In [ ]:
inb = 27
maxb = None
    
fig = plt.figure(figsize=(14,10))
ax = fig.add_subplot(111)
xmarks = np.arange(0.5, corr_matrix[inb:maxb,inb:maxb].shape[0], 1.)
plt.pcolor(corr_matrix[inb:maxb,inb:maxb], cmap='RdBu', vmin=-1, vmax=1)
plt.xticks(xmarks, params[inb:maxb], rotation=90)
plt.yticks(xmarks, params[inb:maxb])
plt.ylim(0,maxb)
plt.xlim(0,maxb)
ax.set_ylim(ax.get_ylim()[::-1])
ax.xaxis.tick_top()
plt.colorbar()
#fig.savefig('/home/jp/projects/atm_nu/ddm_results/prd22_results_may9_v12_final/corr_syst.png', dpi=300)
#fig.savefig('/home/jp/projects/atm_nu/ddm_results/prd22_results_may9_v12_final/corr_syst.pdf')


In [ ]:
# Plots of the stored fit fluxes, ratios directly - does not evaluate splines again
reload(mfp)
mfp.plot_comparison(fit, pullflag=10.,
                   emin=5)

In [ ]:
reload(mfp)

In [ ]:
# Plots of the stored fit fluxes, ratios directly - does not evaluate splines again
f1 = mfp.plotICRC21(fit, data_prefit = True)
#f1.savefig('/home/jp/projects/atm_nu/ddm_results/muon_results/flux_vert_qf2smart3.pdf')

In [ ]:
reload(mfp)
f2 = mfp.plotICRC21(fit, data= 'ratio',data_prefit = True)
#f2.savefig('/home/jp/projects/atm_nu/ddm_results/muon_results/ratio_vert_qf2smart3.pdf')

In [ ]:
# Plots of the stored fit fluxes, ratios directly - does not evaluate splines again
reload(mfp)
f3 = mfp.plotICRC21(fit, data='deis', data_prefit = True)
#f3.savefig('/home/jp/projects/atm_nu/ddm_results/muon_results/flux_deis_qf2smart3.pdf')

In [ ]:
reload(mfp)
f1 = mfp.plotPulls(mufit=fit, pull_scale=20, log_pulls=False
                  )